In [0]:
spark.sql("CREATE VOLUME IF NOT EXISTS dev.demo_db.files")

In [0]:
raw_df = (spark.read
          .format("csv")
          .option("header", "true")
          .option("inferSchema", "true")
          .load("/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv")
          )

In [0]:
raw_df.display()

In [0]:
staging_df = raw_df

In [0]:
#staging_df.write.format("parquet").mode("overwrite").partitionBy("Call Date").save("/Volumes/dev/demo_db/files/fire_calls_tbl")

In [0]:
from pyspark.sql.functions import to_date,to_timestamp,round,year
staging_df = (raw_df.withColumnRenamed("Call Number","CallNumber")
              .withColumnRenamed("Unit ID","UnitID")
              .withColumnRenamed("Incident Number","IncidentNumber")
              .withColumnRenamed("Call Date","CallDate")
              .withColumnRenamed("Watch Date","WatchDate")
              .withColumnRenamed("Call Final Disposition","CallFinalDisposition")
              .withColumnRenamed("Available DtTm","AvailableDtTm")
              .withColumn("Year",year("CallDate")))

In [0]:
staging_df.display()

In [0]:
staging_df.write.format("parquet").mode("overwrite").partitionBy("Year").save("/Volumes/dev/demo_db/files/fire_calls_tbl")

In [0]:
%fs ls /Volumes/dev/demo_db/files/fire_calls_tbl

COnverting into Delta

In [0]:
%sql
CONVERT TO DELTA parquet.`/Volumes/dev/demo_db/files/fire_calls_tbl`
PARTITIONED BY (Year INT)

In [0]:
%sql
describe history delta.`/Volumes/dev/demo_db/files/fire_calls_tbl`